In [2]:
import os
import azureml.core
from azureml.core import (
    Workspace,
    Experiment,
    Dataset,
    Datastore,
    ComputeTarget,
    Environment,
    ScriptRunConfig,
)
from azureml.data import OutputFileDatasetConfig
from azureml.core.compute import AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.core.runconfig import DockerConfiguration

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.49.0


In [3]:
workspace = Workspace.from_config()

In [4]:
exp = Experiment(workspace=workspace, name="nnunet-devops-test")

In [5]:
output = OutputFileDatasetConfig(name="nnUNet", destination=None, source="/output/") # source directory should be same with RESULTS_DIR in docker

In [6]:
#Create an Azure ML Compute:
from azureml.core.compute import AmlCompute

compute_name = "ml-notebook-tst"

if compute_name in workspace.compute_targets:
    compute_target = workspace.compute_targets[compute_name]
    print(f'Found existing compute target {compute_name}')
else:
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC24',
                                                                min_nodes=0,
                                                                max_nodes=1)
    compute_target = ComputeTarget.create(workspace, compute_name, provisioning_config)
    compute_target.wait_for_completion(show_output=True)
    print(f'Compute_target {compute_name} created')

Found existing compute target ml-notebook-tst


In [7]:
# figure out how to mount script folder
script_folder = "/home/azureuser/cloudfiles/code/Users/DSKumar-mda/nnunet/"

prep_step = PythonScriptStep(
    name="prepare step",
    script_name="prepare.py",
    arguments=[], # no arguments for now
    source_directory=script_folder,
    compute_target=compute_target,
    allow_reuse=True,
)

In [8]:
print(compute_target)

AmlCompute(workspace=Workspace.create(name='edp-mlw-mlmm-tst-scentral-01', subscription_id='dc3b2a9f-9106-4946-9d81-3f115060b468', resource_group='databricks-workspace-rg1'), name=ml-notebook-tst, id=/subscriptions/dc3b2a9f-9106-4946-9d81-3f115060b468/resourceGroups/databricks-workspace-rg1/providers/Microsoft.MachineLearningServices/workspaces/edp-mlw-mlmm-tst-scentral-01/computes/ml-notebook-tst, type=AmlCompute, provisioning_state=Succeeded, location=southcentralus, tags={})


In [10]:
docker_env = Environment("nnunet-docker")
docker_env.docker.base_image = "edpcrmlmmtst01.azurecr.io/nnunet:latest" # example : "fastdotai/fastai2:latest"
docker_env.python.user_managed_dependencies = True
docker_config=DockerConfiguration(use_docker=True,shm_size='32g')

train_cfg = ScriptRunConfig(
    source_directory=script_folder,
    script="trainv1.py",
    arguments=[output],
    compute_target=compute_target,
    environment=docker_env,
    docker_runtime_config=docker_config,
)

# create the training step :
train_step = PythonScriptStep(
    name="training step",
    arguments=[],# no asrguments for now, since azure datastore wont work well with colab
    source_directory=train_cfg.source_directory,
    script_name=train_cfg.script,
    runconfig=train_cfg.run_config,
)

In [11]:
pipeline = Pipeline(workspace, steps=[prep_step,train_step])
run = exp.submit(pipeline)

Created step prepare step [84debca1][b97e1538-fb2b-4399-b07f-66d03bad93d9], (This step will run and generate new outputs)
Created step training step [a616cabd][1f1a4706-fc94-46ca-af52-e623f8d32b68], (This step will run and generate new outputs)
Submitted PipelineRun e55a6a4d-fb6a-4142-8c13-fdb0029fd715
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e55a6a4d-fb6a-4142-8c13-fdb0029fd715?wsid=/subscriptions/dc3b2a9f-9106-4946-9d81-3f115060b468/resourcegroups/databricks-workspace-rg1/workspaces/edp-mlw-mlmm-tst-scentral-01&tid=63f2cd41-dedb-4cd2-9130-425526cac16d
